##### Firstly, calculating the average results of 10-flods. <br>
##### Then, we choose three index to analyze results, including delta, UB and IoU. <br>

In [9]:
import os
import time
import torch
import threading
import numpy as np
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
import matplotlib.pyplot as plt

### SetUp

In [43]:
PL_List = ["S1","S2","T1","T2","M0"]
part_List = ["01F","01M","02F","02M","03F","03M","04F","04M","05F","05M"]
mode_List = ["Interflow", "Synchro"]
class_nums2path = {1:"Baseline", 2:"2-Cross", 3:"3-Cross", 4:"4-Cross", 5:"5-Cross"}

### Combination

In [41]:
def Cal_Combination(iter_index=0, Comb_List=[]):
    if iter_index >= len(PL_List):
        if Comb_List!=[]:
            Combination.append(Comb_List)
        return
    # not contain index element
    Cal_Combination(iter_index+1, Comb_List[:])
    # contain index element
    Comb_List_Next = Comb_List[:]
    Comb_List_Next.append(PL_List[iter_index])
    Cal_Combination(iter_index+1, Comb_List_Next[:])

Combination = []
Cal_Combination()
print("cal combination done! total",len(Combination),"combinations.")
Combination.sort(key=lambda x: len(x))
# print(Combination)

cal combination done! total 31 combinations.


### Analyze

#### Average&Delta

In [76]:
# process one mode
for mode in mode_List:
    print("#######################",mode,"#######################")
    # process one combination
    for PL_List in Combination:
        root_path_List = ["./Logs/"+part+"/Analyze/"+mode+"/"+class_nums2path[len(PL_List)]+"/" for part in part_List]
        PLs = "".join(PL_List)
        result_List = []
        for root_path in root_path_List:
            path = root_path+PLs+"_acc.txt"
            df_acc = pd.read_csv(path)
            if mode == "Interflow":
                result_List.append(df_acc.iloc[-1].to_numpy()[0])
            if mode == "Synchro":
                result_List.append(df_acc.sort_values(by=["acc"]).iloc[-1].to_numpy()[0])
        result_average = np.mean(result_List)
        result_std = np.std(result_List)
        print(PLs,":\t",round(result_average,2),"+",round(result_std,2))

####################### Interflow #######################
M0 :	 78.91 + 2.35
T2 :	 68.0 + 3.58
T1 :	 70.02 + 2.21
S2 :	 73.59 + 3.25
S1 :	 76.62 + 3.29
T2M0 :	 77.01 + 2.6
T1M0 :	 79.86 + 3.32
T1T2 :	 70.3 + 2.63
S2M0 :	 80.36 + 2.6
S2T2 :	 76.04 + 2.01
S2T1 :	 77.99 + 2.89
S1M0 :	 80.73 + 2.46
S1T2 :	 76.51 + 2.07
S1T1 :	 78.96 + 3.35
S1S2 :	 78.55 + 3.16
T1T2M0 :	 77.51 + 2.24
S2T2M0 :	 82.42 + 2.29
S2T1M0 :	 82.5 + 2.51
S2T1T2 :	 77.49 + 1.94
S1T2M0 :	 82.23 + 2.15
S1T1M0 :	 82.45 + 2.39
S1T1T2 :	 77.38 + 2.16
S1S2M0 :	 81.19 + 2.72
S1S2T2 :	 81.2 + 2.44
S1S2T1 :	 81.15 + 2.53
S2T1T2M0 :	 82.91 + 2.33
S1T1T2M0 :	 82.65 + 2.37
S1S2T2M0 :	 83.28 + 2.2
S1S2T1M0 :	 83.28 + 2.28
S1S2T1T2 :	 82.39 + 2.27
S1S2T1T2M0 :	 83.84 + 2.13
####################### Synchro #######################
M0 :	 78.91 + 2.35
T2 :	 68.0 + 3.58
T1 :	 70.02 + 2.21
S2 :	 73.59 + 3.25
S1 :	 76.62 + 3.29
T2M0 :	 74.28 + 3.3
T1M0 :	 79.1 + 3.34
T1T2 :	 69.44 + 2.92
S2M0 :	 79.37 + 2.72
S2T2 :	 74.26 + 2.7
S2T1 :	 77

### UB&IoU

##### Setup

In [62]:
part_id2str = {1:"01F",2:"01M",3:"02F",4:"02M",5:"03F",6:"03M",7:"04F",8:"04M",9:"05F",10:"05M"}
part_id = 10
PL_Log_Dict = {
        "S1":"2022-04-25_11-02",
        "S2":"2022-04-25_21-14",
        "T1":"2022-04-26_07-23",
        "T2":"2022-04-26_06-20",
        "M0":"2022-04-24_21-47",
        }
PL_Set_Size = 4

##### Label

In [63]:
label2id = {'e0': 0, 'e1': 1, 'e2': 2, 'e3': 3}
label_path = "./Data/iemocap/iemocap_"+part_id2str[part_id]+".test.csv"
label = np.array(pd.read_csv(label_path)["emotion"].map(lambda emo: label2id[emo]).values)
print("load label done! total",len(label),"samples.")

load label done! total 597 samples.


##### PL Epoch

In [64]:
PL_epoch_dict = {}
# PLset epoch index
for PL in PL_List:
    path = "./Logs/"+part_id2str[part_id]+"/"+PL+"/"+PL_Log_Dict[PL]+"_dev_acc.txt"
    df_acc = pd.read_csv(path, header=None)
    df_acc.columns = ["acc"]
    df_acc.sort_values(by=["acc"], inplace=True, ascending=False)
    PL_epoch_dict[PL] = df_acc.index.to_numpy()[:4]
print("PLset:",PL_epoch_dict)

PLset: {'S1': array([64, 97, 94, 62]), 'S2': array([82, 76, 79, 80]), 'T1': array([31, 96, 10, 35]), 'T2': array([57, 39, 60, 23]), 'M0': array([27, 97, 96, 75])}


##### Comb

In [65]:
def Cal_Epoch_Combination(PL_List=[], Epoch_Combination=[], iter_index=0, Comb_List=[]):
    if iter_index >= len(PL_List):
        if Comb_List != []:
            Epoch_Combination.append(Comb_List)
        return
    for available_out in PL_epoch_dict[PL_List[iter_index]]:
        Comb_List_Next = Comb_List[:]
        Comb_List_Next.append(available_out)
        Cal_Epoch_Combination(PL_List, Epoch_Combination, iter_index+1, Comb_List_Next[:])
    return Epoch_Combination

for PL_List in Combination:
    if len(PL_List)==1:
        continue
    Epoch_Combination_List = Cal_Epoch_Combination(PL_List, Epoch_Combination=[])
    PLs = "".join(PL_List)
    # print("".join(PL_List),"Epoch_Combination numbers:",len(Epoch_Combination_List))
    max_UB = 0
    IoU = 0
    for PLout_comb in Epoch_Combination_List:
        UB = 0
        overlap = 0
        # logits path
        logits_root_path_list = ["./Logs/"+part_id2str[part_id]+"/"+PL_ID+"/Logits/"+PL_Log_Dict[PL_ID]+"/" for PL_ID in PL_List]
        # print(logits_root_path_list)
        logits_path = []
        for PL_index in range(len(PL_List)):  
            logits_path.append(logits_root_path_list[PL_index]+str(PLout_comb[PL_index])+".csv")
        # load logits
        df_logits_list = [pd.read_csv(path, index_col="index")["logits"] for path in logits_path]
        for df_logits in df_logits_list:
            df_logits.sort_index(inplace=True)
        # analyze logits
        for index in range(len(label)):
            logits_list = [np.array(df_logits_list[PL_index][index][1:-1].split(",")).astype(np.float_) for PL_index in range(len(df_logits_list))]
            prediction = np.argmax(logits_list, axis=-1)
            if np.isin(label[index], prediction):
                UB += 1
                _ = np.unique(prediction)
                if len(_)==1:
                    overlap += 1
            if UB>max_UB:
                max_UB = UB
                IoU = overlap/UB
    IoU *= 100
    UB = max_UB
    UBP = 100*max_UB/len(label)
    print(PLs,":\t",UB,"\t",round(UBP,2),"%\t",round(IoU,2),"%")
    analyze_dict = {"UB":max_UB, "UBP":UBP, "IoU":IoU}
    df_analyze = pd.DataFrame.from_dict(analyze_dict, orient="index")
    df_analyze.index.name = "index"
    df_analyze.columns = ["value"]
    df_analyze.to_csv("./Logs/"+part_id2str[part_id]+"/Analyze/Interflow/"+class_nums2path[len(PL_List)]+"/"+PLs+"_analyze.csv")

T2M0 :	 534 	 89.45 %	 63.48 %
T1M0 :	 537 	 89.95 %	 67.23 %
T1T2 :	 493 	 82.58 %	 69.57 %
S2M0 :	 541 	 90.62 %	 68.95 %
S2T2 :	 537 	 89.95 %	 58.66 %
S2T1 :	 545 	 91.29 %	 61.47 %
S1M0 :	 530 	 88.78 %	 70.75 %
S1T2 :	 525 	 87.94 %	 61.14 %
S1T1 :	 536 	 89.78 %	 62.69 %
S1S2 :	 513 	 85.93 %	 71.93 %
T1T2M0 :	 556 	 93.13 %	 53.24 %
S2T2M0 :	 571 	 95.64 %	 49.21 %
S2T1M0 :	 572 	 95.81 %	 51.57 %
S2T1T2 :	 560 	 93.8 %	 49.11 %
S1T2M0 :	 564 	 94.47 %	 51.06 %
S1T1M0 :	 569 	 95.31 %	 53.25 %
S1T1T2 :	 554 	 92.8 %	 49.46 %
S1S2M0 :	 554 	 92.8 %	 59.39 %
S1S2T2 :	 557 	 93.3 %	 49.73 %
S1S2T1 :	 564 	 94.47 %	 53.19 %
S2T1T2M0 :	 580 	 97.15 %	 42.93 %
S1T1T2M0 :	 577 	 96.65 %	 46.27 %
S1S2T2M0 :	 576 	 96.48 %	 44.44 %
S1S2T1M0 :	 580 	 97.15 %	 46.72 %
S1S2T1T2 :	 574 	 96.15 %	 43.9 %
S1S2T1T2M0 :	 585 	 97.99 %	 39.15 %


##### Average 10-flods

In [75]:
for PL_List in Combination:
    if len(PL_List)==1:
        continue
    root_path_List = ["./Logs/"+part+"/Analyze/Interflow/"+class_nums2path[len(PL_List)]+"/" for part in part_List]
    PLs = "".join(PL_List)
    UBP_List = []
    IoU_List = []
    for root_path in root_path_List:
        path = root_path+PLs+"_analyze.csv"
        df_analyze = pd.read_csv(path, index_col="index")
        UBP_List.append(df_analyze.loc["UBP"].to_numpy()[0])
        IoU_List.append(df_analyze.loc["IoU"].to_numpy()[0])
    UBP_average = np.mean(UBP_List)
    IoU_average = np.mean(IoU_List)
    print(PLs,"-->\tUB:",round(UBP_average,2),"%\tIoU:",round(IoU_average,2),"%")

T2M0 -->	UB: 87.05 %	IoU: 63.34 %
T1M0 -->	UB: 87.39 %	IoU: 67.89 %
T1T2 -->	UB: 78.64 %	IoU: 72.18 %
S2M0 -->	UB: 87.57 %	IoU: 70.15 %
S2T2 -->	UB: 87.92 %	IoU: 56.22 %
S2T1 -->	UB: 88.68 %	IoU: 59.41 %
S1M0 -->	UB: 87.45 %	IoU: 73.49 %
S1T2 -->	UB: 87.55 %	IoU: 60.51 %
S1T1 -->	UB: 88.17 %	IoU: 64.01 %
S1S2 -->	UB: 84.63 %	IoU: 73.24 %
T1T2M0 -->	UB: 90.24 %	IoU: 55.38 %
S2T2M0 -->	UB: 92.81 %	IoU: 48.47 %
S2T1M0 -->	UB: 93.02 %	IoU: 52.25 %
S2T1T2 -->	UB: 91.39 %	IoU: 48.05 %
S1T2M0 -->	UB: 92.2 %	IoU: 52.33 %
S1T1M0 -->	UB: 92.51 %	IoU: 55.19 %
S1T1T2 -->	UB: 91.03 %	IoU: 51.53 %
S1S2M0 -->	UB: 91.08 %	IoU: 61.21 %
S1S2T2 -->	UB: 92.17 %	IoU: 48.72 %
S1S2T1 -->	UB: 92.47 %	IoU: 51.8 %
S2T1T2M0 -->	UB: 94.51 %	IoU: 43.34 %
S1T1T2M0 -->	UB: 93.96 %	IoU: 46.97 %
S1S2T2M0 -->	UB: 94.53 %	IoU: 44.33 %
S1S2T1M0 -->	UB: 94.72 %	IoU: 47.64 %
S1S2T1T2 -->	UB: 94.19 %	IoU: 42.37 %
S1S2T1T2M0 -->	UB: 95.81 %	IoU: 40.13 %
